In [1]:
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 17265, done.
remote: Counting objects: 100% (186/186), done.
remote: Compressing objects: 100% (126/126), done.
remote: Total 17265 (delta 107), reused 60 (delta 60), pack-reused 17079 (from 2)
Receiving objects: 100% (17265/17265), 15.93 MiB | 9.81 MiB/s, done.
Resolving deltas: 100% (11813/11813), done.
/content/yolov5


In [2]:
%pwd

'/content/yolov5'

In [3]:
!pip install -qr requirements.txt
import torch

from IPython.display import Image, clear_output

print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 913.5/913.5 kB 22.7 MB/s eta 0:00:00
Setup complete. Using torch 2.5.1+cu121 _CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15102MB, multi_processor_count=40, uuid=6a93d72d-b0aa-18ff-50e5-8cf693ebe42c, L2_cache_size=4MB)


In [4]:
!pwd

/content/yolov5


In [5]:
%cd /content
!curl -L "https://github.com/mirzaumar28/random/raw/refs/heads/main/sign_lang.zip" > SignData.zip; unzip SignData.zip; rm SignData.zip




/content
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 6928k  100 6928k    0     0  4595k      0  0:00:01  0:00:01 --:--:-- 22.3M
Archive:  SignData.zip
  inflating: data.yaml               
  inflating: test/images/Hello_3326a1a2-d9cd-11ef-b94c-c03eba1f5b3d_jpg.rf.803a4df318af7d8944d10d9ab60dfd07.jpg  
  inflating: test/images/Hello_3376b11e-d9cd-11ef-9a72-c03eba1f5b3d_jpg.rf.7a600d831baf86853bbfa73e61c5d539.jpg  
  inflating: test/images/Hello_338925ac-d9cd-11ef-929c-c03eba1f5b3d_jpg.rf.191e1210ad41165de50636fdb0255354.jpg  
  inflating: test/images/Hello_33a8de1e-d9cd-11ef-8c65-c03eba1f5b3d_jpg.rf.f464e879b214270ac2a836da19bcb8a8.jpg  
  inflating: test/images/Hello_342c5726-d9cd-11ef-a776-c03eba1f5b3d_jpg.rf.78acbd492ecbb5382c550c5551a743e0.jpg  
  inflating: test/images/I-Love-You_7de927d2

In [6]:
%cat data.yaml

train: ../train/images
val: ../valid/images
test: ../test/images

nc: 7
names: ['-Hello-Yes-No-Thank-', 'Hello', 'I love You', 'No', 'Please', 'Thank You', 'Yes']

roboflow:
  workspace: object-detection-lirkc
  project: sign-language-uv6r8
  version: 1
  license: CC BY 4.0
  url: https://universe.roboflow.com/object-detection-lirkc/sign-language-uv6r8/dataset/1

In [7]:
import yaml
with open("data.yaml", "r") as stream:
  num_classes = str(yaml.safe_load(stream)['nc'])

In [8]:
num_classes

'7'

In [9]:
%cat /content/yolov5/models/yolov5s.yaml

# Ultralytics 🚀 AGPL-3.0 License - https://ultralytics.com/license

# Parameters
nc: 80 # number of classes
depth_multiple: 0.33 # model depth multiple
width_multiple: 0.50 # layer channel multiple
anchors:
  - [10, 13, 16, 30, 33, 23] # P3/8
  - [30, 61, 62, 45, 59, 119] # P4/16
  - [116, 90, 156, 198, 373, 326] # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [
    [-1, 1, Conv, [64, 6, 2, 2]], # 0-P1/2
    [-1, 1, Conv, [128, 3, 2]], # 1-P2/4
    [-1, 3, C3, [128]],
    [-1, 1, Conv, [256, 3, 2]], # 3-P3/8
    [-1, 6, C3, [256]],
    [-1, 1, Conv, [512, 3, 2]], # 5-P4/16
    [-1, 9, C3, [512]],
    [-1, 1, Conv, [1024, 3, 2]], # 7-P5/32
    [-1, 3, C3, [1024]],
    [-1, 1, SPPF, [1024, 5]], # 9
  ]

# YOLOv5 v6.0 head
head: [
    [-1, 1, Conv, [512, 1, 1]],
    [-1, 1, nn.Upsample, [None, 2, "nearest"]],
    [[-1, 6], 1, Concat, [1]], # cat backbone P4
    [-1, 3, C3, [512, False]], # 13

    [-1, 1, Conv, [256, 1, 1]],
    [-1, 1, nn.Upsample, [None, 2, 

In [10]:
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))


In [11]:
%%writetemplate /content/yolov5/models/custom_yolov5s.yaml

# parameters
nc: {num_classes}  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, BottleneckCSP, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, BottleneckCSP, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, BottleneckCSP, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, BottleneckCSP, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, BottleneckCSP, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, BottleneckCSP, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, BottleneckCSP, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, BottleneckCSP, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

In [12]:
%%time
%cd /content/yolov5/

!python train.py --img 416 --batch 16 --epochs 300 --data '/content/data.yaml' --cfg ./models/custom_yolov5s.yaml --weights 'yolov5s.pt' --name yolov5s_results  --cache

Streaming output truncated to the last 5000 lines.
    165/299         2G    0.02867   0.009289   0.002744         30        416:  38% 6/16 [00:00<00:01,  6.16it/s]/content/yolov5/train.py:412: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp):
    165/299         2G    0.02838   0.008983   0.002797         24        416:  44% 7/16 [00:01<00:01,  6.23it/s]/content/yolov5/train.py:412: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp):
    165/299         2G    0.02822   0.009024   0.002603         28        416:  50% 8/16 [00:01<00:01,  5.41it/s]/content/yolov5/train.py:412: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp):
    165/299         2G    0.02892   0.00

In [13]:
%ls runs/

train/


In [14]:
%ls runs/train/yolov5s_results/weights

best.pt  last.pt


In [15]:
%cd /content/yolov5/
!python detect.py --weights runs/train/yolov5s_results/weights/best.pt --img 416 --conf 0.5 --source ../test/images

/content/yolov5
detect: weights=['runs/train/yolov5s_results/weights/best.pt'], source=../test/images, data=data/coco128.yaml, imgsz=[416, 416], conf_thres=0.5, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_format=0, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-397-gde62f93c Python-3.11.11 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
custom_YOLOv5s summary: 182 layers, 7262700 parameters, 0 gradients
WARNING ⚠️ NMS time limit 0.550s exceeded
image 1/35 /content/test/images/Hello_3326a1a2-d9cd-11ef-b94c-c03eba1f5b3d_jpg.rf.803a4df318af7d8944d10d9ab60dfd07.jpg: 416x416 1 Hello, 7.8ms
image 2/35 /content/test/images/Hello_3376b11e-d9cd-11ef-9a72-c03eba1f5b3d_jpg.rf.7a600d831baf86853bbfa73e61c5d

In [16]:
from google.colab import drive
drive.mount('/content/gdrive')

ValueError: mount failed

In [18]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [22]:
%cp /content/yolov5/runs/train/yolov5s_results/weights/best.pt /content/drive/MyDrive/yoloproject